# Spacy Preprocessing
Get started
```py
conda create -n nlp python=3.9 numpy pandas spacy
conda activate nlp
# for polish
python -m spacy download pl_core_news_lg
# for english
python -m spacy download en_core_web_lg
```

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import spacy
import time

from numpy import ndarray
from pandas import DataFrame, Series
from spacy.tokens.doc import Doc
from spacy.lang.pl import Polish

from typing import List

## Load dataset

In [2]:
dataset: DataFrame = \
    pd.read_csv('polish_sentiment_dataset.csv')
    
print(f'shape: {dataset.shape}')

dataset.head()

shape: (936883, 3)


,description,length,rate
0,Polecam nie pierwszy i nie ostatni raz!,39.0,1.0
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,121.0,1.0
2,Polecam zakupy w tym sklepie. Są dostępne częś...,87.0,1.0
3,0,0.0,0.0
4,Jestem w pełni zadowolona z przebiegu transakcji,48.0,1.0


## Clean dataset

In [3]:
dataset = dataset[:10_000]
dataset = dataset.drop(columns = ['length'])
dataset = dataset[dataset.description.notna()]
dataset = dataset[dataset.rate.notna()]
dataset = dataset[dataset.rate != 0]
dataset = dataset.convert_dtypes()
dataset.dtypes

description    string
rate            Int64
dtype: object

## Initialize spacy

In [4]:
nlp: Polish = \
    spacy.load('pl_core_news_lg')

## Lower text

In [5]:
def lower_token(text: str) -> str:
    return " ".join(token.lemma_ for token in nlp.tokenizer(text))

dataset['lower_tokens'] = \
    dataset.description.apply(lambda text: lower_token(text))

dataset.head()

,description,rate,lower_tokens
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...


## Remove punctuation

In [6]:
def clear_punctuation(text: str) -> str:
    return " ".join(token.text for token in nlp.tokenizer(text) if not token.is_punct)

dataset['tokens_without_punctuation'] = \
    dataset.description.apply(lambda text: clear_punctuation(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...


## Remove stopwords

In [7]:
def clear_stopwords(text: str) -> str:
    return " ".join(token.text for token in nlp.tokenizer(text) if not token.is_stop)

dataset['tokens_without_stopwords'] = \
    dataset.description.apply(lambda text: clear_stopwords(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...


## Get lemma

In [8]:
def get_lemma(text: str) -> str:
    return [token.lemma_ for token in nlp(text, disable=["ner", "parser"])]

dataset['lemma'] = \
    dataset.description.apply(lambda text: get_lemma(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]"
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic..."
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp..."
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran..."
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny..."


## Get chars length

In [9]:
dataset['chars'] = \
    dataset.description.apply(lambda text: len(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86


## Get words length

In [10]:
dataset['words'] = \
    dataset.description.apply(lambda text: len(text.split()))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars,words
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39,7
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121,16
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87,14
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48,7
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86,9


## Get punctuations

In [14]:
dataset['punctuation'] = \
    dataset.description.apply(lambda text: len([token for token in nlp.tokenizer(text) if token.is_punct]))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars,words,part_of_speeches,tags,details,punctuation
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39,7,"[VERB, PART, ADJ, CCONJ, PART, ADJ, NOUN, PUNCT]","[FIN, QUB, ADJ, CONJ, QUB, ADJ, SUBST, INTERP]","[ROOT, advmod, obj, cc, advmod, amod, conj, pu...",1
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121,16,"[ADV, ADJ, NOUN, NOUN, CCONJ, ADJ, PUNCT, VERB...","[ADV, ADJ, SUBST, SUBST, CONJ, ADJ, INTERP, PP...","[advmod, amod, ROOT, nmod, cc, conj, punct, am...",4
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87,14,"[VERB, NOUN, ADP, ADJ, NOUN, PUNCT, VERB, ADJ,...","[FIN, SUBST, PREP, ADJ, SUBST, INTERP, FIN, AD...","[ROOT, obj, case, det, nmod, punct, cop, amod,...",3
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48,7,"[VERB, ADP, NOUN, ADJ, ADP, NOUN, NOUN]","[FIN, PREP, SUBST, ADJ, PREP, SUBST, SUBST]","[cop, case, obl, ROOT, case, obl, nmod]",0
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86,9,"[NOUN, VERB, ADV, PUNCT, ADJ, NOUN, PUNCT, VER...","[SUBST, PRAET, ADV, INTERP, ADJ, SUBST, INTERP...","[nsubj, ROOT, advmod, punct, amod, ROOT, punct...",2


In [15]:
dataset['stopwords'] = \
    dataset.description.apply(lambda text: len([token for token in nlp.tokenizer(text) if token.is_stop]))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars,words,part_of_speeches,tags,details,punctuation,stopwords
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39,7,"[VERB, PART, ADJ, CCONJ, PART, ADJ, NOUN, PUNCT]","[FIN, QUB, ADJ, CONJ, QUB, ADJ, SUBST, INTERP]","[ROOT, advmod, obj, cc, advmod, amod, conj, pu...",1,4
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121,16,"[ADV, ADJ, NOUN, NOUN, CCONJ, ADJ, PUNCT, VERB...","[ADV, ADJ, SUBST, SUBST, CONJ, ADJ, INTERP, PP...","[advmod, amod, ROOT, nmod, cc, conj, punct, am...",4,5
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87,14,"[VERB, NOUN, ADP, ADJ, NOUN, PUNCT, VERB, ADJ,...","[FIN, SUBST, PREP, ADJ, SUBST, INTERP, FIN, AD...","[ROOT, obj, case, det, nmod, punct, cop, amod,...",3,5
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48,7,"[VERB, ADP, NOUN, ADJ, ADP, NOUN, NOUN]","[FIN, PREP, SUBST, ADJ, PREP, SUBST, SUBST]","[cop, case, obl, ROOT, case, obl, nmod]",0,3
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86,9,"[NOUN, VERB, ADV, PUNCT, ADJ, NOUN, PUNCT, VER...","[SUBST, PRAET, ADV, INTERP, ADJ, SUBST, INTERP...","[nsubj, ROOT, advmod, punct, amod, ROOT, punct...",2,2


## Get part of speech

In [11]:
def get_part_of_speech(text: str) -> str:
    return [token.pos_ for token in nlp(text, disable=["ner", "parser"])]

dataset['part_of_speeches'] = \
    dataset.description.apply(lambda text: get_part_of_speech(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars,words,part_of_speeches
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39,7,"[VERB, PART, ADJ, CCONJ, PART, ADJ, NOUN, PUNCT]"
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121,16,"[ADV, ADJ, NOUN, NOUN, CCONJ, ADJ, PUNCT, VERB..."
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87,14,"[VERB, NOUN, ADP, ADJ, NOUN, PUNCT, VERB, ADJ,..."
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48,7,"[VERB, ADP, NOUN, ADJ, ADP, NOUN, NOUN]"
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86,9,"[NOUN, VERB, ADV, PUNCT, ADJ, NOUN, PUNCT, VER..."


## Get tags

In [12]:
def get_tag(text: str) -> str:
    return [token.tag_ for token in nlp(text, disable=["ner", "parser"])]

dataset['tags'] = \
    dataset.description.apply(lambda text: get_tag(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars,words,part_of_speeches,tags
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39,7,"[VERB, PART, ADJ, CCONJ, PART, ADJ, NOUN, PUNCT]","[FIN, QUB, ADJ, CONJ, QUB, ADJ, SUBST, INTERP]"
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121,16,"[ADV, ADJ, NOUN, NOUN, CCONJ, ADJ, PUNCT, VERB...","[ADV, ADJ, SUBST, SUBST, CONJ, ADJ, INTERP, PP..."
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87,14,"[VERB, NOUN, ADP, ADJ, NOUN, PUNCT, VERB, ADJ,...","[FIN, SUBST, PREP, ADJ, SUBST, INTERP, FIN, AD..."
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48,7,"[VERB, ADP, NOUN, ADJ, ADP, NOUN, NOUN]","[FIN, PREP, SUBST, ADJ, PREP, SUBST, SUBST]"
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86,9,"[NOUN, VERB, ADV, PUNCT, ADJ, NOUN, PUNCT, VER...","[SUBST, PRAET, ADV, INTERP, ADJ, SUBST, INTERP..."


## Get details

In [13]:
def get_details(text: str) -> str:
    return [token.dep_ for token in nlp(text, disable=["ner", "tagger"])]

dataset['details'] = \
    dataset.description.apply(lambda text: get_details(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars,words,part_of_speeches,tags,details
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39,7,"[VERB, PART, ADJ, CCONJ, PART, ADJ, NOUN, PUNCT]","[FIN, QUB, ADJ, CONJ, QUB, ADJ, SUBST, INTERP]","[ROOT, advmod, obj, cc, advmod, amod, conj, pu..."
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121,16,"[ADV, ADJ, NOUN, NOUN, CCONJ, ADJ, PUNCT, VERB...","[ADV, ADJ, SUBST, SUBST, CONJ, ADJ, INTERP, PP...","[advmod, amod, ROOT, nmod, cc, conj, punct, am..."
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87,14,"[VERB, NOUN, ADP, ADJ, NOUN, PUNCT, VERB, ADJ,...","[FIN, SUBST, PREP, ADJ, SUBST, INTERP, FIN, AD...","[ROOT, obj, case, det, nmod, punct, cop, amod,..."
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48,7,"[VERB, ADP, NOUN, ADJ, ADP, NOUN, NOUN]","[FIN, PREP, SUBST, ADJ, PREP, SUBST, SUBST]","[cop, case, obl, ROOT, case, obl, nmod]"
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86,9,"[NOUN, VERB, ADV, PUNCT, ADJ, NOUN, PUNCT, VER...","[SUBST, PRAET, ADV, INTERP, ADJ, SUBST, INTERP...","[nsubj, ROOT, advmod, punct, amod, ROOT, punct..."


## Get vector

In [18]:
def get_details(text: str) -> str:
    return [token for token in nlp(text, disable=["ner", "parser", "tagger"])]

dataset['vector'] = \
    dataset.description.apply(lambda text: get_details(text))

dataset.head()

,description,rate,lower_tokens,tokens_without_punctuation,tokens_without_stopwords,lemma,chars,words,part_of_speeches,tags,details,punctuation,stopwords,vector
0,Polecam nie pierwszy i nie ostatni raz!,1,polecam nie pierwszy i nie ostatni raz !,Polecam nie pierwszy i nie ostatni raz,Polecam pierwszy ostatni !,"[polecać, nie, pierwszy, i, nie, ostatni, raz, !]",39,7,"[VERB, PART, ADJ, CCONJ, PART, ADJ, NOUN, PUNCT]","[FIN, QUB, ADJ, CONJ, QUB, ADJ, SUBST, INTERP]","[ROOT, advmod, obj, cc, advmod, amod, conj, pu...",1,4,"[Polecam, nie, pierwszy, i, nie, ostatni, raz, !]"
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,1,bardzo dobra komunikacja sms i telefoniczna . ...,Bardzo dobra komunikacja sms i telefoniczna Za...,dobra komunikacja sms telefoniczna . Zamówiony...,"[bardzo, dobry, komunikacja, sms, i, telefonic...",121,16,"[ADV, ADJ, NOUN, NOUN, CCONJ, ADJ, PUNCT, VERB...","[ADV, ADJ, SUBST, SUBST, CONJ, ADJ, INTERP, PP...","[advmod, amod, ROOT, nmod, cc, conj, punct, am...",4,5,"[Bardzo, dobra, komunikacja, sms, i, telefonic..."
2,Polecam zakupy w tym sklepie. Są dostępne częś...,1,polecam zakupy w tym sklepie . są dostępne czę...,Polecam zakupy w tym sklepie Są dostępne częśc...,Polecam zakupy sklepie . dostępne części stars...,"[polecać, zakup, w, ten, sklep, ., być, dostęp...",87,14,"[VERB, NOUN, ADP, ADJ, NOUN, PUNCT, VERB, ADJ,...","[FIN, SUBST, PREP, ADJ, SUBST, INTERP, FIN, AD...","[ROOT, obj, case, det, nmod, punct, cop, amod,...",3,5,"[Polecam, zakupy, w, tym, sklepie, ., Są, dost..."
4,Jestem w pełni zadowolona z przebiegu transakcji,1,jestem w pełni zadowolona z przebiegu transakcji,Jestem w pełni zadowolona z przebiegu transakcji,pełni zadowolona przebiegu transakcji,"[być, w, pełnia, zadowolony, z, przebieg, tran...",48,7,"[VERB, ADP, NOUN, ADJ, ADP, NOUN, NOUN]","[FIN, PREP, SUBST, ADJ, PREP, SUBST, SUBST]","[cop, case, obl, ROOT, case, obl, nmod]",0,3,"[Jestem, w, pełni, zadowolona, z, przebiegu, t..."
5,Transakcja przebiegła błyskawicznie. Pełen pro...,1,transakcja przebiegła błyskawicznie . pełen pr...,Transakcja przebiegła błyskawicznie Pełen prof...,Transakcja przebiegła błyskawicznie . Pełen pr...,"[transakcja, przebiec, błyskawicznie, ., pełny...",86,9,"[NOUN, VERB, ADV, PUNCT, ADJ, NOUN, PUNCT, VER...","[SUBST, PRAET, ADV, INTERP, ADJ, SUBST, INTERP...","[nsubj, ROOT, advmod, punct, amod, ROOT, punct...",2,2,"[Transakcja, przebiegła, błyskawicznie, ., Peł..."


In [ ]:
# [nlp.vocab['polecam'].norm for i in a]